In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# read files
path = '../../data/WillingToPay/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(4215, 2)


,label,split_text
0,3,今天 天气 很 糟糕 我 需要 帮助 吗 ？
1,2,何时 放弃
2,1,你 说错 了 什么 ？
3,0,是 的 我 已经 准备 好 资金 了
4,2,你们 是 高利贷者 我 借 了 很多 钱 以至于 我 还有 这么 多 东西 我会...
5,2,很少 。
6,3,一个 会 联系
7,1,你 打 错 了 不是 我 借 的
8,1,稍后 再谈
9,2,手上 暂时 没有 这么 多钱 能免 点 吗


In [3]:
data.split_text.iloc[0]

'今天   天气 很 糟糕   我 需要 帮助 吗 ？'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<4215x12416 sparse matrix of type '<class 'numpy.float64'>'
	with 44305 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.978410438909


In [6]:
print(clf.predict(phrase))

[3 2 1 ..., 3 1 2]


# logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.949940688019


In [9]:
print(log_r.predict(phrase))

[3 2 1 ..., 3 1 2]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([3, 2, 1, ..., 3, 1, 2])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1, 1]])

In [15]:
data.label.values

array([3, 2, 1, ..., 3, 1, 2])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 4,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.245314
[10]	valid_0's multi_error: 0.216133
[15]	valid_0's multi_error: 0.194543
[20]	valid_0's multi_error: 0.183393
[25]	valid_0's multi_error: 0.173903
[30]	valid_0's multi_error: 0.164176
[35]	valid_0's multi_error: 0.15516
[40]	valid_0's multi_error: 0.149229
[45]	valid_0's multi_error: 0.144009
[50]	valid_0's multi_error: 0.140214
[55]	valid_0's multi_error: 0.13452
[60]	valid_0's multi_error: 0.129537
[65]	valid_0's multi_error: 0.124318
[70]	valid_0's multi_error: 0.120285
[75]	valid_0's multi_error: 0.116251
[80]	valid_0's multi_error: 0.114828
[85]	valid_0's multi_error: 0.113167
[90]	valid_0's multi_error: 0.111032
[95]	valid_0's multi_error: 0.106762
[100]	valid_0's multi_error: 0.104864
[105]	valid_0's multi_error: 0.103677
[110]	valid_0's multi_error: 0.101305
[115]	valid_0's multi_error: 0.0984579
[120]	valid_0's multi_error: 0.0970344
[125]	valid_0's multi_error: 0.0956109
[130]	

In [17]:
# print(lgbm_model.predict(phrase))

In [18]:
# lgbm_model.predict(phrase)

# Save

In [19]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/WillingToPay/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/WillingToPay/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/WillingToPay/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/WillingToPay/Lgbm.pickle", "wb"))

# Test

In [20]:
test = phrase_vectorizer.transform(['真的 没有 钱'])

In [21]:
clf.predict_proba(test) # linear svc

array([[ 0.00115899,  0.98075249,  0.01162191,  0.00646661]])

In [22]:
log_r.predict_proba(test) # logistic

array([[ 0.02835929,  0.83736505,  0.09742007,  0.03685559]])

In [23]:
lgbm_model.predict(test) # light gbm

array([[ 0.00110875,  0.915063  ,  0.1139374 ,  0.00127664]])

# labeling

In [24]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.980752491219


# save the whole thing

In [25]:
# import jieba
# import numpy as np

# class WillingToPay:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 3
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
from WillingToPay_py import WillingToPay
ifk = WillingToPay(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/WillingToPay/WillingToPay.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.454 seconds.
Prefix dict has been built succesfully.
